In [ ]:
import networkx as nx
import nltk
from operator import itemgetter
import xlrd
import spacy
import string
import os
import matplotlib.pyplot as plt

nltk.download('stopwords')
! python -m spacy download es
nlp = spacy.load("es_core_news_sm")


In [ ]:
def ConstruyeGrafos():
    lista_grafos = []
    nombre_archivo = "NAP.xls"
    libro = xlrd.open_workbook(nombre_archivo)
    hoja = libro.sheet_by_index(0)
    GrafoFrec = nx.Graph()
    GrafoTiempo = nx.Graph()
    GrafoAsoc = nx.Graph()
    num_filas = hoja.nrows
    estimulo = ''
    lema=''
    frecuencia = 0
    tiempo = 0
    asociacion = 0
    for fila in range(0,num_filas-1):
        celda = hoja.cell(fila,0)
        cadena = celda.value
        #print(cadena)
        if cadena == '--PALABRAS--':
            continue
        elif cadena == '======':
            estimulo = hoja.cell(fila+1,4).value
            #doc = nlp(str(estimulo))            
            #estimulo = doc[0].lemma_
        elif cadena == '' or cadena == '=':
            continue
        elif hoja.cell(fila,2).value == '':
            continue
        else: 
            frecuencia = hoja.cell(fila,1).value
            tiempo = hoja.cell(fila,2).value
            asociacion = hoja.cell(fila,3).value
            palabra = hoja.cell(fila,4).value
            estimulo = estimulo.strip()
            #doc = nlp(str(palabra))
            #lema = doc[0].lemma_
            lema = str(palabra)
            lema = lema.strip()
            GrafoFrec.add_edge(estimulo,lema,weight=1/float(frecuencia))
            GrafoTiempo.add_edge(estimulo,lema,weight=float(tiempo))
            GrafoAsoc.add_edge(estimulo,lema,weight=100-float(asociacion))
    lista_grafos.append(GrafoFrec)
    lista_grafos.append(GrafoTiempo)
    lista_grafos.append(GrafoAsoc)  
    #print("Nodos")
    #print(list(GrafoFrec.nodes))
    #print(list(GrafoFrec.edges))
    return lista_grafos


In [ ]:
def limpia_lematiza(cadena):
    limpiado = ''
    
    palabras_funcionales = nltk.corpus.stopwords.words('spanish')
    for c in string.punctuation:
        cadena = cadena.replace(c,"")
    cadena = cadena.strip()
    for palabra in cadena.split(" "):
        if palabra not in palabras_funcionales:
            doc = nlp(palabra)
            limpiado = limpiado + doc[0].lemma_ + " "
    return limpiado

In [ ]:
limpia_lematiza("insecto de rayas que produce miel")

'insecto rayas producir miel '

In [ ]:
def conceptos(lista,subconjunto):
    datos = []
    if len(lista) <= 100:
        tope = len(lista)
    else:
        tope = 100
    for x in range(0,tope):
        if str(lista[x][0]) not in subconjunto and float(lista[x][1])>0:
            datos.append(str(lista[x][0]))
    return datos

In [ ]:
def diccionario_nap(definicion):
    grafos = ConstruyeGrafos()
    grafo_frec = grafos[0]
    grafo_tiempo = grafos[1]
    grafo_asoc = grafos[2]
    texto = limpia_lematiza(definicion)
    texto = texto.lower()
    tokens = texto.split(" ")
    subconjunto_lemas = []
    for palabra in tokens:
        if palabra in grafo_asoc.nodes() and palabra != '':
            subconjunto_lemas.append(palabra)
    #print(subconjunto_lemas)
    if len(subconjunto_lemas) > 0:
        resultados_asociacion = nx.betweenness_centrality_subset(grafo_asoc, subconjunto_lemas, subconjunto_lemas, normalized=True, weight='weight')
        encontrados = sorted(resultados_asociacion.items(),key=itemgetter(1),reverse=True)[0:99]
        print("Asociación")
        print(conceptos(encontrados,subconjunto_lemas))
        
        resultados_frecuencia = nx.betweenness_centrality_subset(grafo_frec, subconjunto_lemas, subconjunto_lemas, normalized=True, weight='weight')
        encontrados = sorted(resultados_frecuencia.items(),key=itemgetter(1),reverse=True)[0:99]
        print("Frecuencia")
        print(conceptos(encontrados,subconjunto_lemas))
        
        resultados_tiempo = nx.betweenness_centrality_subset(grafo_tiempo, subconjunto_lemas, subconjunto_lemas, normalized=True, weight='weight')
        encontrados = sorted(resultados_tiempo.items(),key=itemgetter(1),reverse=True)[0:99]
        print("Tiempo")
        print(conceptos(encontrados,subconjunto_lemas))
        
    else:
        print("Definición muy corta, favor de agregar más datos.")
    return 

In [ ]:
diccionario_nap("Animal conocido como el rey de la selva")

Asociación
['león', 'hipopótamo']
Frecuencia
['león', 'mono']
Tiempo
['león', 'víbora']
